In [1]:
from typing import List
from pydantic import BaseModel, Field

from langchain_google_vertexai import ChatVertexAI

from article.sources import sources
from article.utils import flatten_openapi, generate_extract

import mistune
from IPython.display import Markdown
from tqdm.notebook import tqdm

In [2]:
class Section(BaseModel):
    name: str = Field(title='Name/title of the section')
    points: List[str] = Field(title='List of no more than 3 main points for this section')
    summary: str = Field(title="Summary of the section in 20-50 words")
    experts: List[str] = Field(title="List of 3-5 experts roles/descrptions who can help with article by providing feedback")

    def __str__(self):
        return f'## {self.name}\n{self.summary}\n\n'+','.join(self.points)

class Outline(BaseModel):
    title: str = Field(title="Title of the article")
    summary: str = Field(title="Summary of the article in 20-50 words")
    sections: List[Section]

    def __str__(self):
        return f'# {self.title}\n{self.summary}\n\n' + '\n'.join([str(x) for x in self.sections])

In [3]:
articles = sources()
style_articles = [x[1] for x in articles if x[0].startswith('data/oreilly')]
articles = [x[1] for x in articles]

In [4]:
with open('style.md') as f:
    style = f.read()

### Create outline

In [5]:
initial = """
genai in legacy environment

what is different to startup?
- existing processes, do we want to automate (potenialy inefective process) or optimize
- existing applications - are APIs ready for GenAI?
- old documentation - if I have final1.doc and final2.doc which one has valid info?

even with the the challenges there is tremendous benefits of GenAI in legacy env

What aproaches works:
- start small, focused. Having 100s ideas are great but focus is important
- start with business - what outcomes do we want and how to measure them
- translate it to scenarios - input -> output. with this start evals

Evals
Human -> llm as a judge

Prompt engineering
Split to small steps. Easier to manage, easier to evaluate and easier to "explain" = less black box feeling

Logging
log everything, log authomaticaly. Don't trust applications to log on their side

Security
by default assume not-safe
"""
prompt = [
    ('system', 'You are preparing to write new article. Follow the instruction in the schema. You can chance title and/or provided summary. You sources are' + '\n'.join(articles)),
    ('user', f'Prepare outline and plan for new article. Initial thoughts {initial}')]

In [6]:
g_model = ChatVertexAI(model='gemini-1.5-pro-002', temperature=2, top_k=40, top_n=1)
e_model = ChatVertexAI(model='gemini-1.5-flash-002')

In [7]:
response, outline = generate_extract(
    question=prompt,
    generate_model=g_model,
    schema=Outline,
    extract_model=e_model)

In [8]:
outline.sections[0]

Section(name='Introduction', points=['Hook: Start with a compelling anecdote or statistic about the potential of GenAI.', 'Thesis: GenAI offers significant benefits even in legacy environments, but requires a tailored approach compared to startups.', 'Briefly introduce the key challenges and opportunities (foreshadowing the later sections).'], summary='Introduce the topic and set the stage for the rest of the article.', experts=['Eugene Yan', 'Bryan Bischof', 'Charles Frye', 'Hamel Husain', 'Jason Liu', 'Shreya Shankar'])

In [9]:
Markdown(response)

## Article Outline & Plan: GenAI in a Legacy Environment

**Title:**  Taming the Legacy Beast: Unleashing the Power of Generative AI in Established Organizations

**Summary:** Generative AI offers transformative potential, even within the constraints of legacy systems. This article explores the unique challenges and significant advantages of implementing GenAI in established organizations, providing practical approaches for successful integration.  We'll delve into evaluating existing processes, adapting legacy applications, prompt engineering strategies, evaluation methods, essential logging practices, and crucial security considerations.

**I. Introduction**

* Hook: Start with a compelling anecdote or statistic about the potential of GenAI.
* Thesis:  GenAI offers significant benefits even in legacy environments, but requires a tailored approach compared to startups.
* Briefly introduce the key challenges and opportunities (foreshadowing the later sections).


**II. The Legacy Landscape: Challenges & Opportunities**

* Existing Processes:
    * The temptation (and danger) of automating potentially ineffective processes.
    *  Emphasize the need for optimization before automation. 
    * Case study illustrating a successful vs. unsuccessful automation attempt.
* Legacy Applications & API Readiness:
    *  Discuss common issues with API integration in older systems. 
    * Highlight approaches to overcome these challenges (e.g., wrappers, adapters).
* Documentation and Data Integrity:
    * "final1.doc vs. final2.doc" problem. 
    * Stress the need for data cleaning, validation, and potentially implementing version control systems.


**III.  Practical Approaches for GenAI Implementation**

* Starting Small and Focused:
    * Why a focused approach trumps scattered brainstorming in a legacy setting. 
    * Examples of successful small-scale GenAI projects.
* Business-Driven Development: 
    * Importance of clear business outcomes and measurable goals.
    * Show how business requirements translate to technical scenarios (input/output).
* Translating Business Goals into Scenarios:
    * Detailed examples of how business needs transform into defined scenarios.
    * Demonstrate clear input/output definitions to drive eval development.



**IV.  Evaluation: A Critical Component**

* Human Evaluation in the Loop:
    * Why it remains crucial even with automated methods.
    * Explain different methods of human evaluation: user studies, expert reviews, pairwise comparison.
*  LLM as a Judge:
    * Introduce using LLMs for automated evaluation.
    * Explain its strengths and limitations. 
    * Practical guidelines for using LLMs for effective evaluation.


**V.  Prompt Engineering for Legacy Environments**

* Decomposition and Management:
    * Importance of splitting complex tasks into smaller, manageable prompts. 
    * How smaller prompts simplify evaluation and create a more transparent ("less black box") process.


**VI. Logging:  Leave No Trace Behind**

* The Need for Comprehensive Logging:
    * "Log everything, automatically" principle.  
    * Importance of not relying solely on application-level logging.
    * Discuss potential tooling/frameworks for automated GenAI logging.


**VII.  Security in Legacy + GenAI: A Proactive Approach**

* A "Not-Safe by Default" Mentality:
    * Emphasize starting with strong security assumptions. 
    * Discuss potential vulnerabilities when integrating GenAI with legacy systems.
    * Suggest best practices for threat modeling and mitigating risks (data validation, input sanitization, etc.)



**VIII.  Conclusion**

* Reiterate the benefits and challenges of integrating GenAI in legacy environments. 
* Provide a positive outlook – with careful planning and execution, legacy organizations can successfully harness GenAI for business gains. 
*  Offer a final call to action. 




**Plan:**

1. **Research & Examples:** Gather real-world examples of GenAI projects in legacy environments, successful or otherwise, to illustrate key points.
2. **Technical Depth:**  Ensure adequate technical detail in sections about APIs, logging, and security. Consider code snippets or architecture diagrams for complex scenarios. 
3. **Target Audience:** Tailor the language and examples for the intended audience (e.g., business leaders, developers, or both). 
4. **Clarity and Conciseness:**  Strive for easy-to-understand language and avoid excessive jargon. 
5. **Visuals:** Consider images, charts, or diagrams where helpful.




This comprehensive plan and outline will guide the writing process and create a focused, valuable, and practical article for navigating GenAI integration within the unique complexities of a legacy organizational environment. 


In [15]:
with open('article.md', 'w+') as f:
    f.write(response)